# Tutorial: Hybrid jobs with Qiskit-Braket provider

### Prepare Hybrid job script

In [31]:
! cat ./data/2_hybrid_jobs/job_script.py 

"""Example of Hybrid Job payload with VQE."""
from braket.jobs import save_job_result
from qiskit.opflow import (
    I,
    X,
    Z,
)
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance

from qiskit_braket_provider import BraketLocalBackend


def main():
    backend = BraketLocalBackend(name="default")

    h2_op = (
            (-1.052373245772859 * I ^ I)
            + (0.39793742484318045 * I ^ Z)
            + (-0.39793742484318045 * Z ^ I)
            + (-0.01128010425623538 * Z ^ Z)
            + (0.18093119978423156 * X ^ X)
    )

    quantum_instance = QuantumInstance(
        backend, seed_transpiler=42, seed_simulator=42, shots=10
    )
    ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
    slsqp = SLSQP(maxiter=1)

    vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=quantum_instance)

    vqe_result = vqe.compute_minimum_eigenvalue(h2_o

### Prepare docker image

In [3]:
! cat ./data/2_hybrid_jobs/Dockerfile

FROM 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-base-jobs:1.0-cpu-py37-ubuntu18.04

RUN python3 -m pip install --upgrade pip

RUN python3 -m pip install --no-cache --upgrade git+https://github.com/qiskit-community/qiskit-braket-provider


### Login to AWS CLI and docker

Login to AWS' ERC to pull base image

`aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 292282985366.dkr.ecr.us-west-2.amazonaws.com`

Login to yours ERC to push your image

`aws ecr get-login-password --region <YOUR_AWS_REGION> | docker login --username AWS --password-stdin <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com`

### Create ECR repo

`! aws ecr create-repository --repository-name amazon-braket-qiskit-provider`

### Build docker image

`! docker build -t amazon-braket-qiskit-provider ./data/2_hybrid_jobs/`

`! docker tag amazon-braket-qiskit-provider:latest <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com/amazon-braket-qiskit-provider:latest`

### Submit your job

In [35]:
import uuid

from braket.aws import AwsQuantumJob
from braket.jobs.config import InstanceConfig

image_uri = "<YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com/amazon-braket-qiskit-provider:latest"

job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="data/2_hybrid_jobs/job_script.py",
    entry_point="job_script:main",
    wait_until_complete=False,
    job_name="qiskit-braket-vqe-4",
    image_uri=image_uri
)

In [38]:
from pprint import pprint

pprint(job.result())

{'VQE': {'eigenstate': {'00': 0.7071067811865476,
                        '01': 0.5477225575051661,
                        '11': 0.4472135954999579},
         'eigenvalue': -1.1870890225107222,
         'optimal_parameters': [-1.6508004825151215,
                                4.957957340527567,
                                2.4372737710481633,
                                -3.820554991737687,
                                4.368582921026,
                                4.255770864016807,
                                -2.662412922004604,
                                -4.008723189294315],
         'optimal_point': [-1.6508004825151215,
                           4.957957340527567,
                           2.4372737710481633,
                           -3.820554991737687,
                           4.368582921026,
                           4.255770864016807,
                           -2.662412922004604,
                           -4.008723189294315],
         'optimal_val